In [76]:
import sys
sys.path.append('../../spark-3.1.2-bin-hadoop3.2')
from pyspark.sql.functions import sum
from pyspark.sql import SparkSession

if __name__ == "__main__":
    spark = SparkSession.builder.appName('basic').getOrCreate()
    print('Hi')



Hi


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()

In [3]:
from pyspark.sql.functions import date_format,countDistinct,collect_set,collect_list,concat_ws,col,max,split,initcap,count,length,datediff,expr,avg,round,sum,avg,desc,asc,rank,when,expr,lit,to_date
from pyspark.sql.types import StructType, StructField, IntegerType,StringType,DateType
from pyspark.sql.window import Window



window_spec = Window.partitionBy('race_year')\
    .orderBy(desc('total_points'),desc('wins'))
driverstanding = driver_standing_df.withColumn('rank',rank().over(window_spec))

#### 001 Recyclable and Low Fat Products

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_id  | int     |
| low_fats    | enum    |
| recyclable  | enum    |
+-------------+---------+
product_id is the primary key (column with unique values) for this table.
low_fats is an ENUM (category) of type ('Y', 'N') where 'Y' means this product is low fat and 'N' means it is not.
recyclable is an ENUM (category) of types ('Y', 'N') where 'Y' means this product is recyclable and 'N' means it is not.
 

Write a solution to find the ids of products that are both low fat and recyclable.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Products table:
+-------------+----------+------------+
| product_id  | low_fats | recyclable |
+-------------+----------+------------+
| 0           | Y        | N          |
| 1           | Y        | Y          |
| 2           | N        | Y          |
| 3           | Y        | Y          |
| 4           | N        | N          |
+-------------+----------+------------+
Output: 
+-------------+
| product_id  |
+-------------+
| 1           |
| 3           |
+-------------+
Explanation: Only products 1 and 3 are both low fat and recyclable.

In [17]:
fat_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../data/easymedium/001_RecyclableAndLowFatProducts.csv')


In [18]:
fat_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- low_fats: string (nullable = true)
 |-- recyclable: string (nullable = true)



In [19]:
result_df = fat_df.filter( (col('low_fats') == 'Y') & (col('recyclable') == 'Y') ).select('product_id').orderBy('product_id')


In [18]:
result_df.show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+



In [21]:
fat_df.createTempView('fatdf')

In [23]:
spark.sql("select product_id from fatdf where low_fats ='Y' and recyclable = 'Y' ").show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+



#### 002 Find Customer Referee

Table: Customer

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
| referee_id  | int     |
+-------------+---------+
In SQL, id is the primary key column for this table.
Each row of this table indicates the id of a customer, their name, and the id of the customer who referred them.
 

Find the names of the customer that are not referred by the customer with id = 2.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Customer table:
+----+------+------------+
| id | name | referee_id |
+----+------+------------+
| 1  | Will | null       |
| 2  | Jane | null       |
| 3  | Alex | 2          |
| 4  | Bill | null       |
| 5  | Zack | 1          |
| 6  | Mark | 2          |
+----+------+------------+
Output: 
+------+
| name |
+------+
| Will |
| Jane |
| Bill |
| Zack |
+------+

In [20]:
ref_schema = StructType([
    (StructField('id',IntegerType(),False)),
    (StructField('name',StringType(),False)),
    (StructField('referee_id',IntegerType(),True))
])

In [21]:
ref_df = spark.read.option('header',True).schema(ref_schema).format('csv').load('../data/easymedium/002_FindCustomerFeferre1.csv')


In [65]:
ref_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- referee_id: integer (nullable = true)



In [66]:
ref_df.show()

+---+----+----------+
| id|name|referee_id|
+---+----+----------+
|  1|Will|      null|
|  2|Jane|      null|
|  3|Alex|         2|
|  4|Bill|      null|
|  5|Zack|         1|
|  6|Mark|         2|
+---+----+----------+



In [76]:
ref_df.filter( (col('referee_id') != 2) | (col('referee_id').isNull()) ).select(col('name'),col('referee_id')).show()

+----+----------+
|name|referee_id|
+----+----------+
|Will|      null|
|Jane|      null|
|Bill|      null|
|Zack|         1|
+----+----------+



In [69]:
ref_df.createTempView('refdf1')

In [74]:
spark.sql("select name from refdf1 where  referee_id is null or referee_id !=2").show()

+----+
|name|
+----+
|Will|
|Jane|
|Bill|
|Zack|
+----+



#### 003 Big Countries

Table: World

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| name        | varchar |
| continent   | varchar |
| area        | int     |
| population  | int     |
| gdp         | bigint  |
+-------------+---------+
name is the primary key (column with unique values) for this table.
Each row of this table gives information about the name of a country, the continent to which it belongs, its area, the population, and its GDP value.
 

A country is big if:

it has an area of at least three million (i.e., 3000000 km2), or
it has a population of at least twenty-five million (i.e., 25000000).
Write a solution to find the name, population, and area of the big countries.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
World table:
+-------------+-----------+---------+------------+--------------+
| name        | continent | area    | population | gdp          |
+-------------+-----------+---------+------------+--------------+
| Afghanistan | Asia      | 652230  | 25500100   | 20343000000  |
| Albania     | Europe    | 28748   | 2831741    | 12960000000  |
| Algeria     | Africa    | 2381741 | 37100000   | 188681000000 |
| Andorra     | Europe    | 468     | 78115      | 3712000000   |
| Angola      | Africa    | 1246700 | 20609294   | 100990000000 |
+-------------+-----------+---------+------------+--------------+
Output: 
+-------------+------------+---------+
| name        | population | area    |
+-------------+------------+---------+
| Afghanistan | 25500100   | 652230  |
| Algeria     | 37100000   | 2381741 |
+-------------+------------+---------+


In [22]:
countries_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../data/easymedium/003_BigCountries.csv')


In [86]:
countries_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- continet: string (nullable = true)
 |-- area: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- gdp: long (nullable = true)



In [87]:
countries_df.show()

+-----------+--------+-------+----------+------------+
|       name|continet|   area|population|         gdp|
+-----------+--------+-------+----------+------------+
|Afghanistan|    Asia| 652230|  25500100| 20343000000|
|    Albania|  Europe|  28748|   2831741| 12960000000|
|    Algeria|  Africa|2381741|  37100000|188681000000|
|    Andorra|  Europe|    468|     78115|  3712000000|
|     Angola|  Africa|1246700|  20609294|100990000000|
+-----------+--------+-------+----------+------------+



In [95]:
countries_df.filter( (col('area') >= 3000000) | (col('population') >= 25000000) )\
                    .select(col('name'),col('population'),col('area') ).show()


+-----------+----------+-------+
|       name|population|   area|
+-----------+----------+-------+
|Afghanistan|  25500100| 652230|
|    Algeria|  37100000|2381741|
+-----------+----------+-------+



In [96]:
countries_df.createTempView('countriesdf')

In [97]:
spark.sql("select name,population,area from countriesdf where area >=3000000 OR population >= 25000000 ").show()

+-----------+----------+-------+
|       name|population|   area|
+-----------+----------+-------+
|Afghanistan|  25500100| 652230|
|    Algeria|  37100000|2381741|
+-----------+----------+-------+



#### 004 Article Views

Table: Views

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| article_id    | int     |
| author_id     | int     |
| viewer_id     | int     |
| view_date     | date    |
+---------------+---------+
There is no primary key (column with unique values) for this table, the table may have duplicate rows.
Each row of this table indicates that some viewer viewed an article (written by some author) on some date. 
Note that equal author_id and viewer_id indicate the same person.
 

Write a solution to find all the authors that viewed at least one of their own articles.

Return the result table sorted by id in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Views table:
+------------+-----------+-----------+------------+
| article_id | author_id | viewer_id | view_date  |
+------------+-----------+-----------+------------+
| 1          | 3         | 5         | 2019-08-01 |
| 1          | 3         | 6         | 2019-08-02 |
| 2          | 7         | 7         | 2019-08-01 |
| 2          | 7         | 6         | 2019-08-02 |
| 4          | 7         | 1         | 2019-07-22 |
| 3          | 4         | 4         | 2019-07-21 |
| 3          | 4         | 4         | 2019-07-21 |
+------------+-----------+-----------+------------+
Output: 
+------+
| id   |
+------+
| 4    |
| 7    |
+------+


In [23]:
article_schema = StructType([
    (StructField('article_id',IntegerType(),True)),
    (StructField('author_id',IntegerType(),True)),
    (StructField('viewer_id',IntegerType(),True)),
    (StructField('view_date',DateType(),True))
])

In [24]:
article_df = spark.read.option('header',True).schema(article_schema).format('csv').load('../data/easymedium/004_ArticleViews.csv')


In [106]:
article_df.printSchema()

root
 |-- article_id: integer (nullable = true)
 |-- author_id: integer (nullable = true)
 |-- viewer_id: integer (nullable = true)
 |-- view_date: date (nullable = true)



In [134]:
article_df.filter((col('author_id') == col('viewer_id'))).groupBy('author_id').count()\
        .select('author_id').orderBy(col('author_id').desc()).show()


+---------+
|author_id|
+---------+
|        7|
|        4|
+---------+



In [ ]:
article_df.createTempView('articledf')

In [149]:
spark.sql('select author_id from articledf where author_id = viewer_id group by author_id \
            having count(*) >=1 order by author_id desc' ).show()

+---------+
|author_id|
+---------+
|        7|
|        4|
+---------+



#### 005 Invalid Tweets

Table: Tweets

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| tweet_id       | int     |
| content        | varchar |
+----------------+---------+
tweet_id is the primary key (column with unique values) for this table.
This table contains all the tweets in a social media app.
 

Write a solution to find the IDs of the invalid tweets. The tweet is invalid if the number of characters used in the content of the tweet is strictly greater than 15.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Tweets table:
+----------+----------------------------------+
| tweet_id | content                          |
+----------+----------------------------------+
| 1        | Vote for Biden                   |
| 2        | Let us make America great again! |
+----------+----------------------------------+
Output: 
+----------+
| tweet_id |
+----------+
| 2        |
+----------+
Explanation: 
Tweet 1 has length = 14. It is a valid tweet.
Tweet 2 has length = 32. It is an invalid tweet.

In [25]:
tweets_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../data/easymedium/005_InvalidTweets.csv')


In [155]:
tweets_df.printSchema()

root
 |-- tweet_id: integer (nullable = true)
 |-- content: string (nullable = true)



In [160]:
tweets_df.filter( length(col('content')) > 15).select('tweet_id').show()

+--------+
|tweet_id|
+--------+
|       2|
+--------+



In [161]:
tweets_df.createTempView('tweetsdf')

In [166]:
spark.sql("select tweet_id from tweetsdf where length(content) > 15").show()

+--------+
|tweet_id|
+--------+
|       2|
+--------+



006 Replace Employee ID With The Unique Identifier

In [26]:
employees_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/006_ReplaceEmployeeIDWithTheUniqueIdentifier.csv')


In [27]:
employees_uni_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/006a_ReplaceEmployeeIDWithTheUniqueIdentifier.csv')

In [172]:
employees_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [173]:
employees_uni_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- unique_id: integer (nullable = true)



In [174]:
employees_df.show()

+---+--------+
| id|    name|
+---+--------+
|  1|   Alice|
|  7|     Bob|
| 11|    Meir|
| 90| Winston|
|  3|Jonathan|
+---+--------+



In [175]:
employees_uni_df.show()

+---+---------+
| id|unique_id|
+---+---------+
|  3|        1|
| 11|        2|
| 90|        3|
+---+---------+



In [183]:
employees_df.withColumnRenamed('id','emp_id')\
    .join(employees_uni_df,employees_uni_df.id == col('emp_id'),'left').select('unique_id','name').show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     null|   Alice|
|     null|     Bob|
|        2|    Meir|
|        3| Winston|
|        1|Jonathan|
+---------+--------+



In [184]:
employees_df.createTempView('employeesdf')
employees_uni_df.createTempView('employeesunidf')

In [186]:
spark.sql("select unique_id,name from employeesdf left join employeesunidf on employeesdf.id = employeesunidf.id")\
            .show()


+---------+--------+
|unique_id|    name|
+---------+--------+
|     null|   Alice|
|     null|     Bob|
|        2|    Meir|
|        3| Winston|
|        1|Jonathan|
+---------+--------+



#### 007 Product Sales Analysis

Table: Sales

+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| sale_id     | int   |
| product_id  | int   |
| year        | int   |
| quantity    | int   |
| price       | int   |
+-------------+-------+
(sale_id, year) is the primary key (combination of columns with unique values) of this table.
product_id is a foreign key (reference column) to Product table.
Each row of this table shows a sale on the product product_id in a certain year.
Note that the price is per unit.
 

Table: Product

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| product_id   | int     |
| product_name | varchar |
+--------------+---------+
product_id is the primary key (column with unique values) of this table.
Each row of this table indicates the product name of each product.
 

Write a solution to report the product_name, year, and price for each sale_id in the Sales table.

Return the resulting table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Sales table:
+---------+------------+------+----------+-------+
| sale_id | product_id | year | quantity | price |
+---------+------------+------+----------+-------+ 
| 1       | 100        | 2008 | 10       | 5000  |
| 2       | 100        | 2009 | 12       | 5000  |
| 7       | 200        | 2011 | 15       | 9000  |
+---------+------------+------+----------+-------+
Product table:
+------------+--------------+
| product_id | product_name |
+------------+--------------+
| 100        | Nokia        |
| 200        | Apple        |
| 300        | Samsung      |
+------------+--------------+
Output: 
+--------------+-------+-------+
| product_name | year  | price |
+--------------+-------+-------+
| Nokia        | 2008  | 5000  |
| Nokia        | 2009  | 5000  |
| Apple        | 2011  | 9000  |
+--------------+-------+-------+
Explanation: 
From sale_id = 1, we can conclude that Nokia was sold for 5000 in the year 2008.
From sale_id = 2, we can conclude that Nokia was sold for 5000 in the year 2009.
From sale_id = 7, we can conclude that Apple was sold for 9000 in the year 2011.

In [28]:
sales_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/007_ProductSalesAnalysis.csv')

In [29]:
product_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/007a_ProductSalesAnalysis.csv')

In [191]:
sales_df.printSchema()

root
 |-- sale_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: integer (nullable = true)



In [192]:
product_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)



In [195]:
sales_df.join(product_df,product_df.product_id == sales_df.product_id,'inner')\
    .select('product_name','year','price').show()

+------------+----+-----+
|product_name|year|price|
+------------+----+-----+
|       Nokia|2008| 5000|
|       Nokia|2009| 5000|
|       Apple|2011| 9000|
+------------+----+-----+



In [198]:
sales_df.createOrReplaceTempView('salesdf')
product_df.createTempView('productdf')

In [199]:
spark.sql("select product_name,year,price from salesdf join productdf on salesdf.product_id = productdf.product_id").show()


+------------+----+-----+
|product_name|year|price|
+------------+----+-----+
|       Nokia|2008| 5000|
|       Nokia|2009| 5000|
|       Apple|2011| 9000|
+------------+----+-----+



#### 008  Customer Who Visited but Did Not Make Any Transactions

Table: Visits

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| visit_id    | int     |
| customer_id | int     |
+-------------+---------+
visit_id is the column with unique values for this table.
This table contains information about the customers who visited the mall.
 

Table: Transactions

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| transaction_id | int     |
| visit_id       | int     |
| amount         | int     |
+----------------+---------+
transaction_id is column with unique values for this table.
This table contains information about the transactions made during the visit_id.
 

Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

Return the result table sorted in any order.

The result format is in the following example.

 

Example 1:

Input: 
Visits
+----------+-------------+
| visit_id | customer_id |
+----------+-------------+
| 1        | 23          |
| 2        | 9           |
| 4        | 30          |
| 5        | 54          |
| 6        | 96          |
| 7        | 54          |
| 8        | 54          |
+----------+-------------+
Transactions
+----------------+----------+--------+
| transaction_id | visit_id | amount |
+----------------+----------+--------+
| 2              | 5        | 310    |
| 3              | 5        | 300    |
| 9              | 5        | 200    |
| 12             | 1        | 910    |
| 13             | 2        | 970    |
+----------------+----------+--------+
Output: 
+-------------+----------------+
| customer_id | count_no_trans |
+-------------+----------------+
| 54          | 2              |
| 30          | 1              |
| 96          | 1              |
+-------------+----------------+
Explanation: 
Customer with id = 23 visited the mall once and made one transaction during the visit with id = 12.
Customer with id = 9 visited the mall once and made one transaction during the visit with id = 13.
Customer with id = 30 visited the mall once and did not make any transactions.
Customer with id = 54 visited the mall three times. During 2 visits they did not make any transactions, and during one visit they made 3 transactions.
Customer with id = 96 visited the mall once and did not make any transactions.
As we can see, users with IDs 30 and 96 visited the mall one time without making any transactions. Also, user 54 visited the mall twice and did not make any transactions.

In [31]:
visits_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/008_CustomerWhoVisitedbutDidNotMakeAnyTransactions.csv')

In [32]:
transactiions_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/008a_CustomerWhoVisitedbutDidNotMakeAnyTransactions.csv')

In [205]:
visits_df.printSchema()

root
 |-- visit_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)



In [208]:
visits_df.show()

+--------+-----------+
|visit_id|customer_id|
+--------+-----------+
|       1|         23|
|       2|          9|
|       4|         30|
|       5|         54|
|       6|         96|
|       7|         54|
|       8|         54|
+--------+-----------+



In [209]:
transactiions_df.show()

+--------------+--------+------+
|transaction_id|visit_id|amount|
+--------------+--------+------+
|             2|       5|   310|
|             3|       5|   300|
|             9|       5|   200|
|            12|       1|   910|
|            13|       2|   970|
+--------------+--------+------+



In [206]:
transactiions_df.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- visit_id: integer (nullable = true)
 |-- amount: integer (nullable = true)



In [216]:
visits_df.join(transactiions_df,visits_df.visit_id  == transactiions_df.visit_id,'anti')\
    .groupBy('customer_id').agg(count('*').alias('count')).select('customer_id',col('count')).show()


+-----------+-----+
|customer_id|count|
+-----------+-----+
|         54|    2|
|         96|    1|
|         30|    1|
+-----------+-----+



In [218]:
visits_df.createOrReplaceTempView('visitsdf')
transactiions_df.createOrReplaceTempView('transactiionsdf')

In [220]:
spark.sql("select customer_id,count(*) from visitsdf \
    where visit_id not in (select visit_id from transactiionsdf) \
    group by customer_id").show()

+-----------+--------+
|customer_id|count(1)|
+-----------+--------+
|         54|       2|
|         96|       1|
|         30|       1|
+-----------+--------+



#### 009 Rising Temperature

Table: Weather

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| recordDate    | date    |
| temperature   | int     |
+---------------+---------+
id is the column with unique values for this table.
There are no different rows with the same recordDate.
This table contains information about the temperature on a certain day.
 

Write a solution to find all dates' Id with higher temperatures compared to its previous dates (yesterday).

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Weather table:
+----+------------+-------------+
| id | recordDate | temperature |
+----+------------+-------------+
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |
+----+------------+-------------+
Output: 
+----+
| id |
+----+
| 2  |
| 4  |
+----+
Explanation: 
In 2015-01-02, the temperature was higher than the previous day (10 -> 25).
In 2015-01-04, the temperature was higher than the previous day (20 -> 30).

In [34]:
temperature_schema = StructType([
    (StructField('id',IntegerType(),False)),
    (StructField('recordDate',DateType(),False)),
    (StructField('temperature',IntegerType(),True))
])

In [35]:
temperature_df = spark.read.option('header',True).schema(temperature_schema).format('csv')\
        .load('../data/easymedium/009_RisingTemperature.csv')

In [225]:
temperature_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- recordDate: date (nullable = true)
 |-- temperature: integer (nullable = true)



In [272]:
temperature_df.show()

+---+----------+-----------+
| id|recordDate|temperature|
+---+----------+-----------+
|  1|2015-01-01|         10|
|  2|2015-01-02|         25|
|  3|2015-01-03|         20|
|  4|2015-01-04|         30|
+---+----------+-----------+



In [269]:
w1 = temperature_df
w2 = temperature_df.withColumnRenamed('recordDate','twodate')\
                    .withColumnRenamed('temperature','temp')\
                    .withColumnRenamed('id','temp_id')


In [274]:
w1.join(w2,datediff(w1.recordDate,w2.twodate)==-1).filter(col('temp') > col('temperature'))\
            .select(col('temp_id')).show()


+-------+
|temp_id|
+-------+
|      2|
|      4|
+-------+



In [277]:
temperature_df.createOrReplaceTempView('tempone')
temperature_df.createOrReplaceTempView('recordDate')

In [285]:
spark.sql("select temptwo.id,tempone.recordDate,temptwo.recordDate,temptwo.temperature,tempone.temperature \
        from tempone join temptwo on datediff(tempone.recordDate,temptwo.recordDate)=-1 \
        where temptwo.temperature > tempone.temperature").show()

+---+----------+----------+-----------+-----------+
| id|recordDate|recordDate|temperature|temperature|
+---+----------+----------+-----------+-----------+
|  2|2015-01-01|2015-01-02|         25|         10|
|  4|2015-01-03|2015-01-04|         30|         20|
+---+----------+----------+-----------+-----------+



#### 010 Average Time of Process per Machine

Table: Activity

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| machine_id     | int     |
| process_id     | int     |
| activity_type  | enum    |
| timestamp      | float   |
+----------------+---------+
The table shows the user activities for a factory website.
(machine_id, process_id, activity_type) is the primary key (combination of columns with unique values) of this table.
machine_id is the ID of a machine.
process_id is the ID of a process running on the machine with ID machine_id.
activity_type is an ENUM (category) of type ('start', 'end').
timestamp is a float representing the current time in seconds.
'start' means the machine starts the process at the given timestamp and 'end' means the machine ends the process at the given timestamp.
The 'start' timestamp will always be before the 'end' timestamp for every (machine_id, process_id) pair.
 

There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.

The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the machine_id along with the average time as processing_time, which should be rounded to 3 decimal places.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Activity table:
+------------+------------+---------------+-----------+
| machine_id | process_id | activity_type | timestamp |
+------------+------------+---------------+-----------+
| 0          | 0          | start         | 0.712     |
| 0          | 0          | end           | 1.520     |
| 0          | 1          | start         | 3.140     |
| 0          | 1          | end           | 4.120     |
| 1          | 0          | start         | 0.550     |
| 1          | 0          | end           | 1.550     |
| 1          | 1          | start         | 0.430     |
| 1          | 1          | end           | 1.420     |
| 2          | 0          | start         | 4.100     |
| 2          | 0          | end           | 4.512     |
| 2          | 1          | start         | 2.500     |
| 2          | 1          | end           | 5.000     |
+------------+------------+---------------+-----------+
Output: 
+------------+-----------------+
| machine_id | processing_time |
+------------+-----------------+
| 0          | 0.894           |
| 1          | 0.995           |
| 2          | 1.456           |
+------------+-----------------+
Explanation: 
There are 3 machines running 2 processes each.
Machine 0's average time is ((1.520 - 0.712) + (4.120 - 3.140)) / 2 = 0.894
Machine 1's average time is ((1.550 - 0.550) + (1.420 - 0.430)) / 2 = 0.995
Machine 2's average time is ((4.512 - 4.100) + (5.000 - 2.500)) / 2 = 1.456

In [36]:
machine_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/010_AverageTimeofProcessperMachine.csv')

In [292]:
machine_df.printSchema()

root
 |-- machine_id: integer (nullable = true)
 |-- process_id: integer (nullable = true)
 |-- activity_type: string (nullable = true)
 |-- timestamp: double (nullable = true)



In [342]:
machine_df.show()

+----------+----------+-------------+---------+
|machine_id|process_id|activity_type|timestamp|
+----------+----------+-------------+---------+
|         0|         0|        start|    0.712|
|         0|         0|          end|     1.52|
|         0|         1|        start|     3.14|
|         0|         1|          end|     4.12|
|         1|         0|        start|     0.55|
|         1|         0|          end|     1.55|
|         1|         1|        start|     0.43|
|         1|         1|          end|     1.42|
|         2|         0|        start|      4.1|
|         2|         0|          end|    4.512|
|         2|         1|        start|      2.5|
|         2|         1|          end|      5.0|
+----------+----------+-------------+---------+



In [343]:
start_df = machine_df.filter(col('activity_type') == 'start')

In [344]:
end_df = machine_df.filter(col('activity_type') == 'end').withColumnRenamed('machine_id','end_machine_id')\
        .withColumnRenamed('process_id','end_process_id')\
        .withColumnRenamed('activity_type','end_activity_type')\
        .withColumnRenamed('timestamp','end_timestamp')

In [315]:
start_df.show()

+----------+----------+-------------+---------+
|machine_id|process_id|activity_type|timestamp|
+----------+----------+-------------+---------+
|         0|         0|        start|    0.712|
|         0|         1|        start|     3.14|
|         1|         0|        start|     0.55|
|         1|         1|        start|     0.43|
|         2|         0|        start|      4.1|
|         2|         1|        start|      2.5|
+----------+----------+-------------+---------+



In [316]:
end_df.show()

+--------------+--------------+-----------------+-------------+
|end_machine_id|end_process_id|end_activity_type|end_timestamp|
+--------------+--------------+-----------------+-------------+
|             0|             0|              end|         1.52|
|             0|             1|              end|         4.12|
|             1|             0|              end|         1.55|
|             1|             1|              end|         1.42|
|             2|             0|              end|        4.512|
|             2|             1|              end|          5.0|
+--------------+--------------+-----------------+-------------+



In [341]:
start_df.join(end_df, (start_df.machine_id == end_df.end_machine_id) \
                    & (start_df.process_id == end_df.end_process_id),'inner')\
                    .withColumn('process_time',col('end_timestamp') - col('timestamp'))\
                    .groupby('machine_id')\
                    .agg(round(avg(col('process_time')),3).alias('avg_process_tiime'))\
                    .orderBy(col('machine_id').asc())\
                    .show()
                 

+----------+-----------------+
|machine_id|avg_process_tiime|
+----------+-----------------+
|         0|            0.894|
|         1|            0.995|
|         2|            1.456|
+----------+-----------------+



In [345]:
start_df.createOrReplaceTempView('startdf')
end_df.createOrReplaceTempView('enddf')

In [350]:
spark.sql(" select  machine_id, round(avg(end_timestamp - timestamp),3) as average_process_time \
            from startdf join enddf on machine_id = end_machine_id AND process_id = end_process_id \
            group by machine_id \
            order by machine_id asc" ).show()

+----------+--------------------+
|machine_id|average_process_time|
+----------+--------------------+
|         0|               0.894|
|         1|               0.995|
|         2|               1.456|
+----------+--------------------+



In [351]:
machine_df.createOrReplaceTempView('machinedf')

In [ ]:
select machine_id, round(avg(time_lapse), 3) as processing_time
from
(select machine_id, process_id,
max(timestamp) over (partition by machine_id, process_id) 
-
min(timestamp) over (partition by machine_id, process_id) 
as time_lapse from Activity) temp
group by machine_id

In [358]:
spark.sql(" select machine_id,round(avg(time_lapse),3) as average_processing_time from  (select machine_id,process_id,max(timestamp) OVER(partition by machine_id,process_id) \
                            - min(timestamp) OVER(partition by machine_id,process_id) as time_lapse \
                              from machinedf) temp \
                              group by machine_id \
                              order by machine_id ") .show()

+----------+-----------------------+
|machine_id|average_processing_time|
+----------+-----------------------+
|         0|                  0.894|
|         1|                  0.995|
|         2|                  1.456|
+----------+-----------------------+



#### 011 Employee Bonus

Table: Employee

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| empId       | int     |
| name        | varchar |
| supervisor  | int     |
| salary      | int     |
+-------------+---------+
empId is the column with unique values for this table.
Each row of this table indicates the name and the ID of an employee in addition to their salary and the id of their manager.
 

Table: Bonus

+-------------+------+
| Column Name | Type |
+-------------+------+
| empId       | int  |
| bonus       | int  |
+-------------+------+
empId is the column of unique values for this table.
empId is a foreign key (reference column) to empId from the Employee table.
Each row of this table contains the id of an employee and their respective bonus.
 

Write a solution to report the name and bonus amount of each employee with a bonus less than 1000.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Employee table:
+-------+--------+------------+--------+
| empId | name   | supervisor | salary |
+-------+--------+------------+--------+
| 3     | Brad   | null       | 4000   |
| 1     | John   | 3          | 1000   |
| 2     | Dan    | 3          | 2000   |
| 4     | Thomas | 3          | 4000   |
+-------+--------+------------+--------+
Bonus table:
+-------+-------+
| empId | bonus |
+-------+-------+
| 2     | 500   |
| 4     | 2000  |
+-------+-------+
Output: 
+------+-------+
| name | bonus |
+------+-------+
| Brad | null  |
| John | null  |
| Dan  | 500   |
+------+-------+

In [37]:
emp_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/011_EmployeeBonus.csv')

In [38]:
bonus_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/011a_EmployeeBonus.csv')

In [7]:
emp_df.printSchema()

root
 |-- empId: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- supervisor: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [8]:
bonus_df.printSchema()

root
 |-- empId: integer (nullable = true)
 |-- bonus: integer (nullable = true)



In [17]:
emp_df.join(bonus_df,emp_df.empId == bonus_df.empId,'left')\
        .filter( (col('bonus').isNull()) | (col('bonus') < 1000)).select('name','bonus').show()

+----+-----+
|name|bonus|
+----+-----+
|Brad| null|
|John| null|
| Dan|  500|
+----+-----+



In [18]:
emp_df.createOrReplaceTempView('empdf')
bonus_df.createOrReplaceTempView('bonusdf')

In [23]:
spark.sql("select name,bonus from empdf left join bonusdf on empdf.empId = bonusdf.empId \
        where (bonus is null) OR (bonus < 1000 ) ").show()

+----+-----+
|name|bonus|
+----+-----+
|Brad| null|
|John| null|
| Dan|  500|
+----+-----+



#### 012 Students and Examinations 

Table: Students

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| student_id    | int     |
| student_name  | varchar |
+---------------+---------+
student_id is the primary key (column with unique values) for this table.
Each row of this table contains the ID and the name of one student in the school.
 

Table: Subjects

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| subject_name | varchar |
+--------------+---------+
subject_name is the primary key (column with unique values) for this table.
Each row of this table contains the name of one subject in the school.
 

Table: Examinations

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| student_id   | int     |
| subject_name | varchar |
+--------------+---------+
There is no primary key (column with unique values) for this table. It may contain duplicates.
Each student from the Students table takes every course from the Subjects table.
Each row of this table indicates that a student with ID student_id attended the exam of subject_name.
 

Write a solution to find the number of times each student attended each exam.

Return the result table ordered by student_id and subject_name.

The result format is in the following example.

 

Example 1:

Input: 
Students table:
+------------+--------------+
| student_id | student_name |
+------------+--------------+
| 1          | Alice        |
| 2          | Bob          |
| 13         | John         |
| 6          | Alex         |
+------------+--------------+
Subjects table:
+--------------+
| subject_name |
+--------------+
| Math         |
| Physics      |
| Programming  |
+--------------+
Examinations table:
+------------+--------------+
| student_id | subject_name |
+------------+--------------+
| 1          | Math         |
| 1          | Physics      |
| 1          | Programming  |
| 2          | Programming  |
| 1          | Physics      |
| 1          | Math         |
| 13         | Math         |
| 13         | Programming  |
| 13         | Physics      |
| 2          | Math         |
| 1          | Math         |
+------------+--------------+
Output: 
+------------+--------------+--------------+----------------+
| student_id | student_name | subject_name | attended_exams |
+------------+--------------+--------------+----------------+
| 1          | Alice        | Math         | 3              |
| 1          | Alice        | Physics      | 2              |
| 1          | Alice        | Programming  | 1              |
| 2          | Bob          | Math         | 1              |
| 2          | Bob          | Physics      | 0              |
| 2          | Bob          | Programming  | 1              |
| 6          | Alex         | Math         | 0              |
| 6          | Alex         | Physics      | 0              |
| 6          | Alex         | Programming  | 0              |
| 13         | John         | Math         | 1              |
| 13         | John         | Physics      | 1              |
| 13         | John         | Programming  | 1              |
+------------+--------------+--------------+----------------+
Explanation: 
The result table should contain all students and all subjects.
Alice attended the Math exam 3 times, the Physics exam 2 times, and the Programming exam 1 time.
Bob attended the Math exam 1 time, the Programming exam 1 time, and did not attend the Physics exam.
Alex did not attend any exams.
John attended the Math exam 1 time, the Physics exam 1 time, and the Programming exam 1 time.

In [39]:
student_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/012_StudentsandExaminations.csv')\
        .withColumnRenamed('student_id','stu_student_id')

In [40]:
subject_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/012a_StudentsandExaminations.csv')

In [41]:
examp_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/012b_StudentsandExaminations.csv')\
        .withColumnRenamed('student_id','ex_student_id') \
        .withColumnRenamed('subject_name','ex_subject_name')

In [56]:
student_df.printSchema()

root
 |-- stu_student_id: integer (nullable = true)
 |-- student_name: string (nullable = true)



In [47]:
subject_df.printSchema()

root
 |-- subject_name: string (nullable = true)



In [48]:
examp_df.printSchema()

root
 |-- ex_student_id: integer (nullable = true)
 |-- ex_subject_name: string (nullable = true)



In [95]:
examp_df.join(student_df,col('ex_student_id') == col('stu_student_id'),'inner')\
        .join(subject_df,col('ex_subject_name') == col('subject_name'),'inner')\
        .groupBy('stu_student_id','student_name','ex_subject_name')\
        .agg(count('ex_subject_name').alias('attended_exames'))\
        .select('stu_student_id','student_name','ex_subject_name',col('attended_exames'))\
        .orderBy(col('stu_student_id'),col('ex_subject_name')).show()

+--------------+------------+---------------+---------------+
|stu_student_id|student_name|ex_subject_name|attended_exames|
+--------------+------------+---------------+---------------+
|             1|       Alice|           Math|              3|
|             1|       Alice|        Physics|              2|
|             1|       Alice|    Programming|              1|
|             2|         Bob|           Math|              1|
|             2|         Bob|    Programming|              1|
|            13|        John|           Math|              1|
|            13|        John|        Physics|              1|
|            13|        John|    Programming|              1|
+--------------+------------+---------------+---------------+



In [116]:
student_df.join(subject_df)\
.show()

+--------------+------------+------------+
|stu_student_id|student_name|subject_name|
+--------------+------------+------------+
|             1|       Alice|        Math|
|             1|       Alice|     Physics|
|             1|       Alice| Programming|
|             2|         Bob|        Math|
|             2|         Bob|     Physics|
|             2|         Bob| Programming|
|            13|        John|        Math|
|            13|        John|     Physics|
|            13|        John| Programming|
|             6|        Alex|        Math|
|             6|        Alex|     Physics|
|             6|        Alex| Programming|
+--------------+------------+------------+



In [137]:
student_df.join(subject_df)\
    .join(examp_df, (col('ex_student_id') == col('stu_student_id'))
          & (col('subject_name') == col('ex_subject_name') ),'left')\
        .groupBy('stu_student_id','student_name','subject_name','ex_subject_name')\
        .agg(count('ex_subject_name').alias('attended_exames'))\
        .select('stu_student_id','student_name','subject_name',col('attended_exames'))\
        .orderBy(col('stu_student_id'),col('subject_name')).show()


+--------------+------------+------------+---------------+
|stu_student_id|student_name|subject_name|attended_exames|
+--------------+------------+------------+---------------+
|             1|       Alice|        Math|              3|
|             1|       Alice|     Physics|              2|
|             1|       Alice| Programming|              1|
|             2|         Bob|        Math|              1|
|             2|         Bob|     Physics|              0|
|             2|         Bob| Programming|              1|
|             6|        Alex|        Math|              0|
|             6|        Alex|     Physics|              0|
|             6|        Alex| Programming|              0|
|            13|        John|        Math|              1|
|            13|        John|     Physics|              1|
|            13|        John| Programming|              1|
+--------------+------------+------------+---------------+



In [172]:
student_df.createOrReplaceTempView('studentdf')
subject_df.createOrReplaceTempView('subjectdf')
examp_df.createOrReplaceTempView('exampdf')
spark.sql("select stu_student_id,student_name,subject_name,count(ex_subject_name) as attended_exames\
        from  studentdf outer join subjectdf \
        left join exampdf on stu_student_id = ex_student_id AND subject_name = ex_subject_name\
        group by stu_student_id,student_name,subject_name \
        order by stu_student_id,subject_name")\
        .show(50)

+--------------+------------+------------+---------------+
|stu_student_id|student_name|subject_name|attended_exames|
+--------------+------------+------------+---------------+
|             1|       Alice|        Math|              3|
|             1|       Alice|     Physics|              2|
|             1|       Alice| Programming|              1|
|             2|         Bob|        Math|              1|
|             2|         Bob|     Physics|              0|
|             2|         Bob| Programming|              1|
|             6|        Alex|        Math|              0|
|             6|        Alex|     Physics|              0|
|             6|        Alex| Programming|              0|
|            13|        John|        Math|              1|
|            13|        John|     Physics|              1|
|            13|        John| Programming|              1|
+--------------+------------+------------+---------------+



#### 013 Not Boring Movies

Table: Cinema

+----------------+----------+
| Column Name    | Type     |
+----------------+----------+
| id             | int      |
| movie          | varchar  |
| description    | varchar  |
| rating         | float    |
+----------------+----------+
id is the primary key (column with unique values) for this table.
Each row contains information about the name of a movie, its genre, and its rating.
rating is a 2 decimal places float in the range [0, 10]
 

Write a solution to report the movies with an odd-numbered ID and a description that is not "boring".

Return the result table ordered by rating in descending order.

The result format is in the following example.

 

Example 1:

Input: 
Cinema table:
+----+------------+-------------+--------+
| id | movie      | description | rating |
+----+------------+-------------+--------+
| 1  | War        | great 3D    | 8.9    |
| 2  | Science    | fiction     | 8.5    |
| 3  | irish      | boring      | 6.2    |
| 4  | Ice song   | Fantacy     | 8.6    |
| 5  | House card | Interesting | 9.1    |
+----+------------+-------------+--------+
Output: 
+----+------------+-------------+--------+
| id | movie      | description | rating |
+----+------------+-------------+--------+
| 5  | House card | Interesting | 9.1    |
| 1  | War        | great 3D    | 8.9    |
+----+------------+-------------+--------+
Explanation: 
We have three movies with odd-numbered IDs: 1, 3, and 5. The movie with ID = 3 is boring so we do not include it in the answer.



In [42]:
movies_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/013_NotBoringMovies.csv')

In [175]:
movies_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- movie: string (nullable = true)
 |-- description: string (nullable = true)
 |-- rating: double (nullable = true)



In [182]:
movies_df.filter( (col('id')%2 == 1) & (col('description') != 'boring') )\
    .select('*').orderBy(col('rating').desc()).show()

+---+----------+-----------+------+
| id|     movie|description|rating|
+---+----------+-----------+------+
|  5|House card|Interesting|   9.1|
|  1|       War|   great 3D|   8.9|
+---+----------+-----------+------+



In [178]:
movies_df.createOrReplaceTempView('moviesdf')

In [180]:
spark.sql("select * from moviesdf \
        where id%2 = 1 AND description != 'boring' \
        order by rating desc").show()

+---+----------+-----------+------+
| id|     movie|description|rating|
+---+----------+-----------+------+
|  5|House card|Interesting|   9.1|
|  1|       War|   great 3D|   8.9|
+---+----------+-----------+------+



#### 014 Average Selling Price  

Table: Prices

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| start_date    | date    |
| end_date      | date    |
| price         | int     |
+---------------+---------+
(product_id, start_date, end_date) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates the price of the product_id in the period from start_date to end_date.
For each product_id there will be no two overlapping periods. That means there will be no two intersecting periods for the same product_id.
 

Table: UnitsSold

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| purchase_date | date    |
| units         | int     |
+---------------+---------+
This table may contain duplicate rows.
Each row of this table indicates the date, units, and product_id of each product sold. 
 

Write a solution to find the average selling price for each product. average_price should be rounded to 2 decimal places.

Return the result table in any order.

The result format is in the following example.

 
Example 1:

Input: 
Prices table:
+------------+------------+------------+--------+
| product_id | start_date | end_date   | price  |
+------------+------------+------------+--------+
| 1          | 2019-02-17 | 2019-02-28 | 5      |
| 1          | 2019-03-01 | 2019-03-22 | 20     |
| 2          | 2019-02-01 | 2019-02-20 | 15     |
| 2          | 2019-02-21 | 2019-03-31 | 30     |
+------------+------------+------------+--------+
UnitsSold table:
+------------+---------------+-------+
| product_id | purchase_date | units |
+------------+---------------+-------+
| 1          | 2019-02-25    | 100   |
| 1          | 2019-03-01    | 15    |
| 2          | 2019-02-10    | 200   |
| 2          | 2019-03-22    | 30    |
+------------+---------------+-------+
Output: 
+------------+---------------+
| product_id | average_price |
+------------+---------------+
| 1          | 6.96          |
| 2          | 16.96         |
+------------+---------------+
Explanation: 
Average selling price = Total Price of Product / Number of products sold.
Average selling price for product 1 = ((100 * 5) + (15 * 20)) / 115 = 6.96
Average selling price for product 2 = ((200 * 15) + (30 * 30)) / 230 = 16.96

In [44]:
avgprice_schema = StructType([
    (StructField('product_id',IntegerType(),False)),
    (StructField('start_date',DateType(),False)),
    (StructField('end_date',DateType(),False)),
    (StructField('price',IntegerType(),True))
])

In [45]:
avgprice_df = spark.read.option('header',True).schema(avgprice_schema).format('csv')\
        .load('../data/easymedium/014_AverageSellingPrice.csv')

In [46]:
sold_schema = StructType([
    (StructField('product_id',IntegerType(),False)),
    (StructField('purchase_date',DateType(),False)),
    (StructField('units',IntegerType(),True))
])

In [47]:
sold_df = spark.read.option('header',True).schema(sold_schema).format('csv')\
        .load('../data/easymedium/014a_AverageSellingPrice.csv')\
        .withColumnRenamed('product_id','sol_product_id')

In [278]:
avgprice_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- start_date: date (nullable = true)
 |-- end_date: date (nullable = true)
 |-- price: integer (nullable = true)



In [220]:
sold_df.printSchema()

root
 |-- sol_product_id: integer (nullable = true)
 |-- purchase_date: date (nullable = true)
 |-- units: integer (nullable = true)



In [194]:
avgprice_df.show()

+----------+----------+----------+-----+
|product_id|start_date|  end_date|price|
+----------+----------+----------+-----+
|         1|2019-02-17|2019-02-28|    5|
|         1|2019-03-01|2019-03-22|   20|
|         2|2019-02-01|2019-02-20|   15|
|         2|2019-02-21|2019-03-31|   30|
+----------+----------+----------+-----+



In [290]:
avgprice_df.join(sold_df,col('product_id') == col('sol_product_id'),'inner')\
    .filter(col('purchase_date').between(col('start_date'),col('end_date')))\
    .withColumn('mul', col('price') * col('units'))\
    .show()

+----------+----------+----------+-----+--------------+-------------+-----+----+
|product_id|start_date|  end_date|price|sol_product_id|purchase_date|units| mul|
+----------+----------+----------+-----+--------------+-------------+-----+----+
|         1|2019-02-17|2019-02-28|    5|             1|   2019-02-25|  100| 500|
|         1|2019-03-01|2019-03-22|   20|             1|   2019-03-01|   15| 300|
|         2|2019-02-01|2019-02-20|   15|             2|   2019-02-10|  200|3000|
|         2|2019-02-21|2019-03-31|   30|             2|   2019-03-22|   30| 900|
+----------+----------+----------+-----+--------------+-------------+-----+----+



In [291]:
avgprice_df.join(sold_df,col('product_id') == col('sol_product_id'),'inner')\
    .filter(col('purchase_date').between(col('start_date'),col('end_date')))\
    .withColumn('mul', col('price') * col('units'))\
    .groupby('product_id')\
    .agg( round(sum(col('mul'))/sum(col('units')),3).alias('avg_price') )\
    .show()

+----------+---------+
|product_id|avg_price|
+----------+---------+
|         1|    6.957|
|         2|   16.957|
+----------+---------+



In [280]:
sold_df.createOrReplaceTempView('solddf')
avgprice_df.createOrReplaceTempView('avgpricedf')

In [310]:
spark.sql(" select avg.product_id, \
              round(sum( (avg.price * sol.units) ) / sum(sol.units),3) as avg_price \
              from avgpricedf avg join solddf sol on avg.product_id = sol.sol_product_id \
              where sol.purchase_date between avg.start_date and avg.end_date \
              group by   avg.product_id \
                  ").show()

+----------+---------+
|product_id|avg_price|
+----------+---------+
|         1|    6.957|
|         2|   16.957|
+----------+---------+



#### 015 Project Employees

Table: Project

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| project_id  | int     |
| employee_id | int     |
+-------------+---------+
(project_id, employee_id) is the primary key of this table.
employee_id is a foreign key to Employee table.
Each row of this table indicates that the employee with employee_id is working on the project with project_id.
 

Table: Employee

+------------------+---------+
| Column Name      | Type    |
+------------------+---------+
| employee_id      | int     |
| name             | varchar |
| experience_years | int     |
+------------------+---------+
employee_id is the primary key of this table. It's guaranteed that experience_years is not NULL.
Each row of this table contains information about one employee.
 

Write an SQL query that reports the average experience years of all the employees for each project, rounded to 2 digits.

Return the result table in any order.

The query result format is in the following example.

 

Example 1:

Input: 
Project table:
+-------------+-------------+
| project_id  | employee_id |
+-------------+-------------+
| 1           | 1           |
| 1           | 2           |
| 1           | 3           |
| 2           | 1           |
| 2           | 4           |
+-------------+-------------+
Employee table:
+-------------+--------+------------------+
| employee_id | name   | experience_years |
+-------------+--------+------------------+
| 1           | Khaled | 3                |
| 2           | Ali    | 2                |
| 3           | John   | 1                |
| 4           | Doe    | 2                |
+-------------+--------+------------------+
Output: 
+-------------+---------------+
| project_id  | average_years |
+-------------+---------------+
| 1           | 2.00          |
| 2           | 2.50          |
+-------------+---------------+
Explanation: The average experience years for the first project is (3 + 2 + 1) / 3 = 2.00 and for the second project is (3 + 2) / 2 = 2.50

In [48]:
project_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/015_ProjectEmployees.csv')\
        .withColumnRenamed('employee_id','pro_employee_id')

In [323]:
project_df.printSchema()

root
 |-- project_id: integer (nullable = true)
 |-- pro_employee_id: integer (nullable = true)



In [49]:
emp_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/015a_ProjectEmployees.csv')

In [325]:
emp_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- experience_years: integer (nullable = true)



In [333]:
project_df.join(emp_df,col('pro_employee_id') == col('employee_id'), 'inner' ).groupby('project_id') \
        .agg(avg(col('experience_years')).alias('avg_years')).show()

+----------+---------+
|project_id|avg_years|
+----------+---------+
|         1|      2.0|
|         2|      2.5|
+----------+---------+



In [334]:
project_df.createOrReplaceTempView('projectdf')
emp_df.createOrReplaceTempView('empdf')

In [339]:
spark.sql("select project_id,avg(experience_years) as avg_years \
              from projectdf join empdf on pro_employee_id = employee_id \
              group by project_id ").show()
            

+----------+---------+
|project_id|avg_years|
+----------+---------+
|         1|      2.0|
|         2|      2.5|
+----------+---------+



#### 016 Percentage of Users Attended a Contest   

Table: Users

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| user_id     | int     |
| user_name   | varchar |
+-------------+---------+
user_id is the primary key (column with unique values) for this table.
Each row of this table contains the name and the id of a user.
 

Table: Register

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| contest_id  | int     |
| user_id     | int     |
+-------------+---------+
(contest_id, user_id) is the primary key (combination of columns with unique values) for this table.
Each row of this table contains the id of a user and the contest they registered into.
 

Write a solution to find the percentage of the users registered in each contest rounded to two decimals.

Return the result table ordered by percentage in descending order. In case of a tie, order it by contest_id in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Users table:
+---------+-----------+
| user_id | user_name |
+---------+-----------+
| 6       | Alice     |
| 2       | Bob       |
| 7       | Alex      |
+---------+-----------+
Register table:
+------------+---------+
| contest_id | user_id |
+------------+---------+
| 215        | 6       |
| 209        | 2       |
| 208        | 2       |
| 210        | 6       |
| 208        | 6       |
| 209        | 7       |
| 209        | 6       |
| 215        | 7       |
| 208        | 7       |
| 210        | 2       |
| 207        | 2       |
| 210        | 7       |
+------------+---------+
Output: 
+------------+------------+
| contest_id | percentage |
+------------+------------+
| 208        | 100.0      |
| 209        | 100.0      |
| 210        | 100.0      |
| 215        | 66.67      |
| 207        | 33.33      |
+------------+------------+
Explanation: 
All the users registered in contests 208, 209, and 210. The percentage is 100% and we sort them in the answer table by contest_id in ascending order.
Alice and Alex registered in contest 215 and the percentage is ((2/3) * 100) = 66.67%
Bob registered in contest 207 and the percentage is ((1/3) * 100) = 33.33%


In [50]:
user_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/016_PercentageofUsersAttendedaContest.csv')\
        .withColumnRenamed('user_id','use_user_id')

In [51]:
contest_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/016a_PercentageofUsersAttendedaContest.csv')

In [351]:
user_df.printSchema()

root
 |-- use_user_id: integer (nullable = true)
 |-- user_name: string (nullable = true)



In [343]:
contest_df.printSchema()

root
 |-- contest_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)



In [437]:
result_df = user_df.join(contest_df,col('use_user_id') == col('user_id'),'inner' ) \
        .groupby('contest_id') \
        .agg( round(((count('use_user_id')/3) * 100 ),2).alias('percentage') ) \
        .orderBy(desc('percentage'),asc('contest_id')) \
        .show()

+----------+----------+
|contest_id|percentage|
+----------+----------+
|       208|     100.0|
|       209|     100.0|
|       210|     100.0|
|       215|     66.67|
|       207|     33.33|
+----------+----------+



In [378]:
user_df.createOrReplaceTempView('userdf')
contest_df.createOrReplaceTempView('contestdf')

In [431]:
spark.sql(" select contest_id,percentage from \
          (select contest_id, round(((count('use_user_id')/3) * 100 ),2) as percentage \
          from userdf join  contestdf on use_user_id = user_id \
          group by contest_id) a  \
          order by percentage desc, contest_id asc ").show()

+----------+----------+
|contest_id|percentage|
+----------+----------+
|       208|     100.0|
|       209|     100.0|
|       210|     100.0|
|       215|     66.67|
|       207|     33.33|
+----------+----------+



#### 017 Queries Quality and Percentage 

Table: Queries

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| query_name  | varchar |
| result      | varchar |
| position    | int     |
| rating      | int     |
+-------------+---------+
This table may have duplicate rows.
This table contains information collected from some queries on a database.
The position column has a value from 1 to 500.
The rating column has a value from 1 to 5. Query with rating less than 3 is a poor query.
 

We define query quality as:

The average of the ratio between query rating and its position.

We also define poor query percentage as:

The percentage of all queries with rating less than 3.

Write a solution to find each query_name, the quality and poor_query_percentage.

Both quality and poor_query_percentage should be rounded to 2 decimal places.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Queries table:
+------------+-------------------+----------+--------+
| query_name | result            | position | rating |
+------------+-------------------+----------+--------+
| Dog        | Golden Retriever  | 1        | 5      |
| Dog        | German Shepherd   | 2        | 5      |
| Dog        | Mule              | 200      | 1      |
| Cat        | Shirazi           | 5        | 2      |
| Cat        | Siamese           | 3        | 3      |
| Cat        | Sphynx            | 7        | 4      |
+------------+-------------------+----------+--------+
Output: 
+------------+---------+-----------------------+
| query_name | quality | poor_query_percentage |
+------------+---------+-----------------------+
| Dog        | 2.50    | 33.33                 |
| Cat        | 0.66    | 33.33                 |
+------------+---------+-----------------------+
Explanation: 
Dog queries quality is ((5 / 1) + (5 / 2) + (1 / 200)) / 3 = 2.50
Dog queries poor_ query_percentage is (1 / 3) * 100 = 33.33

Cat queries quality equals ((2 / 5) + (3 / 3) + (4 / 7)) / 3 = 0.66
Cat queries poor_ query_percentage is (1 / 3) * 100 = 33.33

In [52]:
query_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/017_QueriesQualityandPercentage.csv')

In [441]:
query_df.printSchema()

root
 |-- query_name: string (nullable = true)
 |-- result: string (nullable = true)
 |-- position: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [684]:
q_df = query_df.groupby('query_name')\
        .agg(round(avg(col('rating')/col('position') ),3).alias('quality')) \
        .withColumn('q_percentage',lit(0))

In [53]:
quality_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/017_QueriesQualityandPercentage.csv')

In [638]:
x = quality_df.groupby('query_name').agg(count(col('query_name')).alias('cnt')).withColumn('q_percentage',lit(0)) 

In [639]:
y = quality_df.filter(col('rating') < 3).groupby('query_name').agg(count(col('query_name')).alias('cnt'))

In [640]:
yy = y.withColumn('q_percentage',lit(0)).select('query_name','q_percentage','cnt')

In [659]:
xy_union_df  = x.union(yy)

In [681]:
percentage_df = xy_union_df.groupby('query_name')\
    .agg(sum(col('cnt')).alias('cnt'), sum(col('q_percentage')).alias('q_percentage'))\
    .withColumn('q_percentage',(col('q_percentage')/col('cnt'))*100)\
    .select('query_name',lit(0).alias('quality'),'q_percentage')\
    

In [688]:
q_df.union(percentage_df).groupby('query_name')\
    .agg(sum(col('quality')).alias('quality'),round(sum(col('q_percentage')),2).alias('q_percentage')) \
    .show()

+----------+-------+------------+
|query_name|quality|q_percentage|
+----------+-------+------------+
|       Cat|  0.657|       33.33|
|       Dog|  2.502|       33.33|
+----------+-------+------------+



In [632]:
query_df.createOrReplaceTempView('querydf')

In [564]:
spark.sql(" select query_name,sum(query) as query, round(sum(q_percentage),2) as q_percentage from \
             (select query_name, round(avg(rating/position),3) as query,0 as q_percentage \
              from querydf group by query_name \
            UNION \
            ( \
              select query_name,0 as quality,(sum(q_pct)/sum(cnt))*100 as q_percentage from (\
            ( select query_name,count(query_name) as cnt,0 as q_pct from querydf \
              group by query_name )\
            union \
            ( select query_name, 0 as cnt,count(rating) as rat from querydf \
              where rating < 3  group by query_name )) \
              group by query_name \
            ) \
            ) group by query_name \
            ").show()

+----------+-----+------------+
|query_name|query|q_percentage|
+----------+-----+------------+
|       Cat|0.657|       33.33|
|       Dog|2.502|       33.33|
+----------+-----+------------+



#### 018 Number of Unique Subjects Taught by Each Teacher

Table: Teacher

+-------------+------+
| Column Name | Type |
+-------------+------+
| teacher_id  | int  |
| subject_id  | int  |
| dept_id     | int  |
+-------------+------+
(subject_id, dept_id) is the primary key (combinations of columns with unique values) of this table.
Each row in this table indicates that the teacher with teacher_id teaches the subject subject_id in the department dept_id.
 

Write a solution to calculate the number of unique subjects each teacher teaches in the university.

Return the result table in any order.

The result format is shown in the following example.

 

Example 1:

Input: 
Teacher table:
+------------+------------+---------+
| teacher_id | subject_id | dept_id |
+------------+------------+---------+
| 1          | 2          | 3       |
| 1          | 2          | 4       |
| 1          | 3          | 3       |
| 2          | 1          | 1       |
| 2          | 2          | 1       |
| 2          | 3          | 1       |
| 2          | 4          | 1       |
+------------+------------+---------+
Output:  
+------------+-----+
| teacher_id | cnt |
+------------+-----+
| 1          | 2   |
| 2          | 4   |
+------------+-----+
Explanation: 
Teacher 1:
  - They teach subject 2 in departments 3 and 4.
  - They teach subject 3 in department 3.
Teacher 2:
  - They teach subject 1 in department 1.
  - They teach subject 2 in department 1.
  - They teach subject 3 in department 1.
  - They teach subject 4 in department 1.

In [54]:
subject_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/018_NumberofUniqueSubjectsTaughtbyEachTeacher.csv')

In [690]:
subject_df.printSchema()

root
 |-- teacher_id: integer (nullable = true)
 |-- subject_id: integer (nullable = true)
 |-- dept_id: integer (nullable = true)



In [698]:
subject_df.select('teacher_id','subject_id').distinct().groupby('teacher_id').count().show()

+----------+-----+
|teacher_id|count|
+----------+-----+
|         1|    2|
|         2|    4|
+----------+-----+



In [699]:
subject_df.createOrReplaceTempView('subjectdf')

In [700]:
spark.sql("select teacher_id, count(distinct subject_id) as count from subjectdf\
            group by teacher_id").show()

+----------+-----+
|teacher_id|count|
+----------+-----+
|         1|    2|
|         2|    4|
+----------+-----+



#### 019 User Activity for the Past 30 Days  

Table: Activity

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| session_id    | int     |
| activity_date | date    |
| activity_type | enum    |
+---------------+---------+
This table may have duplicate rows.
The activity_type column is an ENUM (category) of type ('open_session', 'end_session', 'scroll_down', 'send_message').
The table shows the user activities for a social media website. 
Note that each session belongs to exactly one user.
 

Write a solution to find the daily active user count for a period of 30 days ending 2019-07-27 inclusively. A user was active on someday if they made at least one activity on that day.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Activity table:
+---------+------------+---------------+---------------+
| user_id | session_id | activity_date | activity_type |
+---------+------------+---------------+---------------+
| 1       | 1          | 2019-07-20    | open_session  |
| 1       | 1          | 2019-07-20    | scroll_down   |
| 1       | 1          | 2019-07-20    | end_session   |
| 2       | 4          | 2019-07-20    | open_session  |
| 2       | 4          | 2019-07-21    | send_message  |
| 2       | 4          | 2019-07-21    | end_session   |
| 3       | 2          | 2019-07-21    | open_session  |
| 3       | 2          | 2019-07-21    | send_message  |
| 3       | 2          | 2019-07-21    | end_session   |
| 4       | 3          | 2019-06-25    | open_session  |
| 4       | 3          | 2019-06-25    | end_session   |
+---------+------------+---------------+---------------+
Output: 
+------------+--------------+ 
| day        | active_users |
+------------+--------------+ 
| 2019-07-20 | 2            |
| 2019-07-21 | 2            |
+------------+--------------+ 
Explanation: Note that we do not care about days with zero active users.

In [55]:
useract_Schama = sold_schema =  StructType([
                                (StructField('user_id',IntegerType(),False)),
                                (StructField('session_id',IntegerType(),False)),
                                (StructField('activity_date',DateType(),False)),
                                (StructField('activity_type',StringType(),True))
                            ])

In [56]:
#Write a solution to find the daily active user count for a period of 30 days ending 2019-07-27 inclusively.
useract_df = spark.read.option('header',True).schema(useract_Schama).format('csv')\
        .load('../data/easymedium/019_UserActivityforthePast30Days.csv')

In [57]:
useract_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- session_id: integer (nullable = true)
 |-- activity_date: date (nullable = true)
 |-- activity_type: string (nullable = true)



In [30]:
useract_df.filter(datediff(to_date(lit('2019-07-27') ),col('activity_date')) < 30)\
        .groupby('activity_date').agg(countDistinct('user_id').alias('active_users')).show()

+-------------+------------+
|activity_date|active_users|
+-------------+------------+
|   2019-07-21|           2|
|   2019-07-20|           2|
+-------------+------------+



In [31]:
useract_df.createOrReplaceTempView('useractdf')

In [41]:
spark.sql("select activity_date, count(distinct user_id) from useractdf \
                where DATEDIFF(to_date('2019-07-27' ),activity_date) < 30 \
                group by activity_date").show()

+-------------+-----------------------+
|activity_date|count(DISTINCT user_id)|
+-------------+-----------------------+
|   2019-07-21|                      2|
|   2019-07-20|                      2|
+-------------+-----------------------+



#### 020 Classes More Than 5 Students

Table: Courses

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| student     | varchar |
| class       | varchar |
+-------------+---------+
(student, class) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates the name of a student and the class in which they are enrolled.
 

Write a solution to find all the classes that have at least five students.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Courses table:
+---------+----------+
| student | class    |
+---------+----------+
| A       | Math     |
| B       | English  |
| C       | Math     |
| D       | Biology  |
| E       | Math     |
| F       | Computer |
| G       | Math     |
| H       | Math     |
| I       | Math     |
+---------+----------+
Output: 
+---------+
| class   |
+---------+
| Math    |
+---------+
Explanation: 
- Math has 6 students, so we include it.
- English has 1 student, so we do not include it.
- Biology has 1 student, so we do not include it.
- Computer has 1 student, so we do not include it.

In [58]:
studentt_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/020_ClassesMoreThan5Students.csv')

In [43]:
studentt_df.printSchema()

root
 |-- student: string (nullable = true)
 |-- class: string (nullable = true)



In [49]:
studentt_df.groupby('class').agg(count(col('class')).alias('cnt')).filter(col('cnt')>=5).select(col('class')).show()

+-----+
|class|
+-----+
| Math|
+-----+



In [53]:
studentt_df.createOrReplaceTempView('studenttdf')

In [54]:
spark.sql("select class from studenttdf group by class having count(class) >=5 ").show()

+-----+
|class|
+-----+
| Math|
+-----+



#### 021 Find Followers Count|

Table: Followers

+-------------+------+
| Column Name | Type |
+-------------+------+
| user_id     | int  |
| follower_id | int  |
+-------------+------+
(user_id, follower_id) is the primary key (combination of columns with unique values) for this table.
This table contains the IDs of a user and a follower in a social media app where the follower follows the user.
 

Write a solution that will, for each user, return the number of followers.

Return the result table ordered by user_id in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Followers table:
+---------+-------------+
| user_id | follower_id |
+---------+-------------+
| 0       | 1           |
| 1       | 0           |
| 2       | 0           |
| 2       | 1           |
+---------+-------------+
Output: 
+---------+----------------+
| user_id | followers_count|
+---------+----------------+
| 0       | 1              |
| 1       | 1              |
| 2       | 2              |
+---------+----------------+
Explanation: 
The followers of 0 are {1}
The followers of 1 are {0}
The followers of 2 are {0,1}

In [59]:
followers_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/021_Find_Followers_Count.csv')

In [56]:
followers_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- follower_id: integer (nullable = true)



In [59]:
followers_df.groupby('user_id').agg(count('follower_id').alias('follower_id')).orderBy('user_id').show()

+-------+-----------+
|user_id|follower_id|
+-------+-----------+
|      0|          1|
|      1|          1|
|      2|          2|
+-------+-----------+



In [60]:
followers_df.createOrReplaceTempView('followersdf')

In [61]:
spark.sql("select user_id,count(follower_id) as follower_id from  followersdf group by user_id order by user_id").show()

+-------+-----------+
|user_id|follower_id|
+-------+-----------+
|      0|          1|
|      1|          1|
|      2|          2|
+-------+-----------+



#### 022 Biggest Single Number

Table: MyNumbers

+-------------+------+
| Column Name | Type |
+-------------+------+
| num         | int  |
+-------------+------+
This table may contain duplicates (In other words, there is no primary key for this table in SQL).
Each row of this table contains an integer.
 

A single number is a number that appeared only once in the MyNumbers table.

Find the largest single number. If there is no single number, report null.

The result format is in the following example.

 

Example 1:

Input: 
MyNumbers table:
+-----+
| num |
+-----+
| 8   |
| 8   |
| 3   |
| 3   |
| 1   |
| 4   |
| 5   |
| 6   |
+-----+
Output: 
+-----+
| num |
+-----+
| 6   |
+-----+
Explanation: The single numbers are 1, 4, 5, and 6.
Since 6 is the largest single number, we return it.
Example 2:

Input: 
MyNumbers table:
+-----+
| num |
+-----+
| 8   |
| 8   |
| 7   |
| 7   |
| 3   |
| 3   |
| 3   |
+-----+
Output: 
+------+
| num  |
+------+
| null |
+------+
Explanation: There are no single numbers in the input table so we return null.

In [60]:
single_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/022_BiggestSingleNumber.csv')

In [63]:
single_df.printSchema()

root
 |-- num: integer (nullable = true)



In [79]:
single_df.groupby('num').agg(count('num').alias('cnt'))\
        .filter(col('cnt') == 1).select(max('num')).show()

+--------+
|max(num)|
+--------+
|       6|
+--------+



In [80]:
single_df.createOrReplaceTempView('singledf')

In [96]:
spark.sql(" select max(m_num) from (select max(num) as m_num from singledf group by num having count(num) = 1 )").show()

+----------+
|max(m_num)|
+----------+
|         6|
+----------+



#### 023 The Number of Employees Which Report to Each Employee  

Table: Employees

+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| reports_to  | int      |
| age         | int      |
+-------------+----------+
employee_id is the column with unique values for this table.
This table contains information about the employees and the id of the manager they report to. Some employees do not report to anyone (reports_to is null). 
 

For this problem, we will consider a manager an employee who has at least 1 other employee reporting to them.

Write a solution to report the ids and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by employee_id.

The result format is in the following example.

 

Example 1:

Input: 
Employees table:
+-------------+---------+------------+-----+
| employee_id | name    | reports_to | age |
+-------------+---------+------------+-----+
| 9           | Hercy   | null       | 43  |
| 6           | Alice   | 9          | 41  |
| 4           | Bob     | 9          | 36  |
| 2           | Winston | null       | 37  |
+-------------+---------+------------+-----+
Output: 
+-------------+-------+---------------+-------------+
| employee_id | name  | reports_count | average_age |
+-------------+-------+---------------+-------------+
| 9           | Hercy | 2             | 39          |
+-------------+-------+---------------+-------------+
Explanation: Hercy has 2 people report directly to him, Alice and Bob. Their average age is (41+36)/2 = 38.5, which is 39 after rounding it to the nearest integer.
Example 2:

Input: 
Employees table:
+-------------+---------+------------+-----+ 
| employee_id | name    | reports_to | age |
|-------------|---------|------------|-----|
| 1           | Michael | null       | 45  |
| 2           | Alice   | 1          | 38  |
| 3           | Bob     | 1          | 42  |
| 4           | Charlie | 2          | 34  |
| 5           | David   | 2          | 40  |
| 6           | Eve     | 3          | 37  |
| 7           | Frank   | null       | 50  |
| 8           | Grace   | null       | 48  |
+-------------+---------+------------+-----+ 
Output: 
+-------------+---------+---------------+-------------+
| employee_id | name    | reports_count | average_age |
| ----------- | ------- | ------------- | ----------- |
| 1           | Michael | 2             | 40          |
| 2           | Alice   | 2             | 37          |
| 3           | Bob     | 1             | 37          |
+-------------+---------+---------------+-------------+


In [61]:
mng_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/023a_The NumberofEmployeesWhichReportoEach Employee.csv')

In [169]:
mng_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- reports_to: integer (nullable = true)
 |-- age: integer (nullable = true)



In [170]:
name_df = mng_df.select('employee_id','name')

In [171]:
mng_df.filter(col('reports_to').isNotNull()).groupby('reports_to')\
    .agg(count('reports_to').alias('report_count'),avg('age').alias('avg_age'))\
    .join(name_df,col('employee_id') == col('reports_to'),'inner')\
    .select('employee_id','name','report_count',round('avg_age',0).alias('avg_age')).show()

+-----------+-------+------------+-------+
|employee_id|   name|report_count|avg_age|
+-----------+-------+------------+-------+
|          1|Michael|           2|   40.0|
|          2|  Alice|           2|   37.0|
|          3|    Bob|           1|   37.0|
+-----------+-------+------------+-------+



In [172]:
mng_df.createOrReplaceTempView('mngdf')

In [173]:
spark.sql(" select employee_id,name,report_count,round(avg_age,0) as avg_age from mngdf \
            join ( \
            select reports_to,count(reports_to) as report_count,avg(age) as avg_age \
            from mngdf where reports_to is not null group by reports_to ) a on employee_id = a.reports_to \
          ").show()

+-----------+-------+------------+-------+
|employee_id|   name|report_count|avg_age|
+-----------+-------+------------+-------+
|          1|Michael|           2|   40.0|
|          2|  Alice|           2|   37.0|
|          3|    Bob|           1|   37.0|
+-----------+-------+------------+-------+



#### 024 Primary Department for Each Employee

Table: Employee

+---------------+---------+
| Column Name   |  Type   |
+---------------+---------+
| employee_id   | int     |
| department_id | int     |
| primary_flag  | varchar |
+---------------+---------+
(employee_id, department_id) is the primary key (combination of columns with unique values) for this table.
employee_id is the id of the employee.
department_id is the id of the department to which the employee belongs.
primary_flag is an ENUM (category) of type ('Y', 'N'). If the flag is 'Y', the department is the primary department for the employee. If the flag is 'N', the department is not the primary.
 

Employees can belong to multiple departments. When the employee joins other departments, they need to decide which department is their primary department. Note that when an employee belongs to only one department, their primary column is 'N'.

Write a solution to report all the employees with their primary department. For employees who belong to one department, report their only department.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Employee table:
+-------------+---------------+--------------+
| employee_id | department_id | primary_flag |
+-------------+---------------+--------------+
| 1           | 1             | N            |
| 2           | 1             | Y            |
| 2           | 2             | N            |
| 3           | 3             | N            |
| 4           | 2             | N            |
| 4           | 3             | Y            |
| 4           | 4             | N            |
+-------------+---------------+--------------+
Output: 
+-------------+---------------+
| employee_id | department_id |
+-------------+---------------+
| 1           | 1             |
| 2           | 1             |
| 3           | 3             |
| 4           | 3             |
+-------------+---------------+
Explanation: 
- The Primary department for employee 1 is 1.
- The Primary department for employee 2 is 1.
- The Primary department for employee 3 is 3.
- The Primary department for employee 4 is 3.

In [62]:
primemp_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/024_PrimaryDepartmentforEachEmployee.csv')

In [177]:
primemp_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- primary_flag: string (nullable = true)



In [192]:
primemp_df.filter( (col('primary_flag') == 'Y')).select('employee_id','department_id')\
    .union( primemp_df.groupby('employee_id').agg(count(col('department_id')).alias('cnt_dep')) \
    .filter(col('cnt_dep') == 1)).orderBy('employee_id').show()

+-----------+-------------+
|employee_id|department_id|
+-----------+-------------+
|          1|            1|
|          2|            1|
|          3|            1|
|          4|            3|
+-----------+-------------+



In [193]:
primemp_df.createOrReplaceTempView('primemp_df')

In [208]:
spark.sql("select employee_id,department_id from primemp_df where primary_flag = 'Y' \
          UNION ( select employee_id, count(department_id) as department_id from primemp_df \
          group by employee_id having department_id = 1 ) ORDER BY employee_id ").show()

+-----------+-------------+
|employee_id|department_id|
+-----------+-------------+
|          1|            1|
|          2|            1|
|          3|            1|
|          4|            3|
+-----------+-------------+



#### 025 Triangle Judgement

Table: Triangle

+-------------+------+
| Column Name | Type |
+-------------+------+
| x           | int  |
| y           | int  |
| z           | int  |
+-------------+------+
In SQL, (x, y, z) is the primary key column for this table.
Each row of this table contains the lengths of three line segments.
 

Report for every three line segments whether they can form a triangle.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Triangle table:
+----+----+----+
| x  | y  | z  |
+----+----+----+
| 13 | 15 | 30 |
| 10 | 20 | 15 |
+----+----+----+
Output: 
+----+----+----+----------+
| x  | y  | z  | triangle |
+----+----+----+----------+
| 13 | 15 | 30 | No       |
| 10 | 20 | 15 | Yes      |
+----+----+----+----------+

In [63]:
triangle_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/025_TriangleJudgement.csv')

In [211]:
triangle_df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- z: integer (nullable = true)



In [246]:
triangle_df.select('x','y','z', when( ((col('x')+ col('y' ) ) > col('z') ) \
                                   &  ((col('x')+ col('y' ) ) > col('z') ) \
                                   &  ((col('x')+ col('y' ) ) > col('z') ),'YES').otherwise('NO').alias('triangle')).show()

+---+---+---+--------+
|  x|  y|  z|triangle|
+---+---+---+--------+
| 13| 15| 30|      NO|
| 10| 20| 15|     YES|
+---+---+---+--------+



In [247]:
### WRONG
triangle_df.select('x','y','z', when( (col('x')+ col('y' ) ) > col('z'),'YES' ) \
                               .when( (col('z')+ col('y' ) ) > col('x'),'YES' ) \
                               .when( (col('x')+ col('z' ) ) > col('y'),'YES' ).otherwise('NO')\
                                .alias('triangle')).show()


+---+---+---+--------+
|  x|  y|  z|triangle|
+---+---+---+--------+
| 13| 15| 30|     YES|
| 10| 20| 15|     YES|
+---+---+---+--------+



In [237]:
triangle_df.createOrReplaceTempView('triangle_df')

In [243]:
spark.sql("SELECT x, y, z, \
           CASE WHEN (x + y) > z AND (x + z) > y AND (y + z) > x THEN 'YES' ELSE 'NO' END AS triangle \
    FROM triangle_df").show()

+---+---+---+--------+
|  x|  y|  z|triangle|
+---+---+---+--------+
| 13| 15| 30|      NO|
| 10| 20| 15|     YES|
+---+---+---+--------+



#### 026  Employees Whose Manager Left the Company   

Table: Employees

+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| manager_id  | int      |
| salary      | int      |
+-------------+----------+
In SQL, employee_id is the primary key for this table.
This table contains information about the employees, their salary, and the ID of their manager. Some employees do not have a manager (manager_id is null). 
 

Find the IDs of the employees whose salary is strictly less than $30000 and whose manager left the company. When a manager leaves the company, their information is deleted from the Employees table, but the reports still have their manager_id set to the manager that left.

Return the result table ordered by employee_id.

The result format is in the following example.

 

Example 1:

Input:  
Employees table:
+-------------+-----------+------------+--------+
| employee_id | name      | manager_id | salary |
+-------------+-----------+------------+--------+
| 3           | Mila      | 9          | 60301  |
| 12          | Antonella | null       | 31000  |
| 13          | Emery     | null       | 67084  |
| 1           | Kalel     | 11         | 21241  |
| 9           | Mikaela   | null       | 50937  |
| 11          | Joziah    | 6          | 28485  |
+-------------+-----------+------------+--------+
Output: 
+-------------+
| employee_id |
+-------------+
| 11          |
+-------------+

Explanation: 
The employees with a salary less than $30000 are 1 (Kalel) and 11 (Joziah).
Kalel's manager is employee 11, who is still in the company (Joziah).
Joziah's manager is employee 6, who left the company because there is no row for employee 6 as it was deleted.

In [64]:
lftcomp = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/026_EmployeesWhoseManagerLefttheCompany.csv')

In [249]:
lftcomp.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [305]:
emp_df = lftcomp.withColumnRenamed('employee_id','emp_id').select(col('emp_id'))

In [306]:
gr3000_df = lftcomp.filter( col('salary') < 30000)

In [304]:
gr3000_df.show()

+-----------+------+----------+------+
|employee_id|  name|manager_id|salary|
+-----------+------+----------+------+
|          1| Kalel|        11| 21241|
|         11|Joziah|         6| 28485|
+-----------+------+----------+------+



In [312]:
gr3000_df.join(emp_df,col('manager_id') == col('emp_id'),'left')\
    .filter(col('emp_id').isNull() ).select('employee_id','name','salary').show()

+-----------+------+------+
|employee_id|  name|salary|
+-----------+------+------+
|         11|Joziah| 28485|
+-----------+------+------+



In [313]:
lftcomp.createOrReplaceTempView('lftcompdf')

In [325]:
spark.sql(" select a.employee_id from ( \
          (select employee_id,name,manager_id,salary from lftcompdf where salary < 30000) as a \
          left join (select employee_id from  lftcompdf ) b on a.manager_id = b.employee_id ) \
          where b.employee_id is null").show()

+-----------+
|employee_id|
+-----------+
|         11|
+-----------+



#### 027 Fix Names in a Table

Table: Users

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| user_id        | int     |
| name           | varchar |
+----------------+---------+
user_id is the primary key (column with unique values) for this table.
This table contains the ID and the name of the user. The name consists of only lowercase and uppercase characters.
 

Write a solution to fix the names so that only the first character is uppercase and the rest are lowercase.

Return the result table ordered by user_id.

The result format is in the following example.

 

Example 1:

Input: 
Users table:
+---------+-------+
| user_id | name  |
+---------+-------+
| 1       | aLice |
| 2       | bOB   |
+---------+-------+
Output: 
+---------+-------+
| user_id | name  |
+---------+-------+
| 1       | Alice |
| 2       | Bob   |
+---------+-------+

In [65]:
caps_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/027_FixNamesinaTable.csv')

In [327]:
caps_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [332]:
caps_df.select('user_id',initcap('name').alias('name')).show()

+-------+-----+
|user_id| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
+-------+-----+



In [331]:
caps_df.createOrReplaceTempView('capsdf')

In [335]:
spark.sql("select user_id,initcap(name) as name from capsdf").show()

+-------+-----+
|user_id| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
+-------+-----+



#### 028 Patients With a Condition

Table: Patients

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| patient_id   | int     |
| patient_name | varchar |
| conditions   | varchar |
+--------------+---------+
patient_id is the primary key (column with unique values) for this table.
'conditions' contains 0 or more code separated by spaces. 
This table contains information of the patients in the hospital.
 

Write a solution to find the patient_id, patient_name, and conditions of the patients who have Type I Diabetes. Type I Diabetes always starts with DIAB1 prefix.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Patients table:
+------------+--------------+--------------+
| patient_id | patient_name | conditions   |
+------------+--------------+--------------+
| 1          | Daniel       | YFEV COUGH   |
| 2          | Alice        |              |
| 3          | Bob          | DIAB100 MYOP |
| 4          | George       | ACNE DIAB100 |
| 5          | Alain        | DIAB201      |
+------------+--------------+--------------+
Output: 
+------------+--------------+--------------+
| patient_id | patient_name | conditions   |
+------------+--------------+--------------+
| 3          | Bob          | DIAB100 MYOP |
| 4          | George       | ACNE DIAB100 | 
+------------+--------------+--------------+
Explanation: Bob and George both have a condition that starts with DIAB1.

In [66]:
pat_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/028_PatientsWithaCondition.csv')

In [338]:
pat_df.printSchema()

root
 |-- patient_id: integer (nullable = true)
 |-- patient_name: string (nullable = true)
 |-- conditions: string (nullable = true)



In [379]:
pat_df.withColumn('col1',split(col('conditions'),' ')[0]) \
.withColumn('col2',split(col('conditions'),' ')[1]) \
.filter( (col('col1').like('DIAB1%')) | (col('col2').like('DIAB1%')) ) \
.select('patient_id','patient_name','conditions').show()

+----------+------------+------------+
|patient_id|patient_name|  conditions|
+----------+------------+------------+
|         3|         Bob|DIAB100 MYOP|
|         4|      George|ACNE DIAB100|
+----------+------------+------------+



In [380]:
pat_df.createOrReplaceTempView('patdf')

In [388]:
spark.sql("select patient_id,patient_name,conditions from \
          (select patient_id,patient_name,conditions,split(conditions,' ')[0] as col1, split(conditions,' ')[1] as col2 from patdf ) \
          where col1 like 'DIAB1%' OR col2 like 'DIAB1%' ").show()



+----------+------------+------------+
|patient_id|patient_name|  conditions|
+----------+------------+------------+
|         3|         Bob|DIAB100 MYOP|
|         4|      George|ACNE DIAB100|
+----------+------------+------------+



#### 029  Delete Duplicate Emails

Table: Person

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| email       | varchar |
+-------------+---------+
id is the primary key (column with unique values) for this table.
Each row of this table contains an email. The emails will not contain uppercase letters.
 

Write a solution to delete all duplicate emails, keeping only one unique email with the smallest id.

For SQL users, please note that you are supposed to write a DELETE statement and not a SELECT one.

For Pandas users, please note that you are supposed to modify Person in place.

After running your script, the answer shown is the Person table. The driver will first compile and run your piece of code and then show the Person table. The final order of the Person table does not matter.

The result format is in the following example.

 

Example 1:

Input: 
Person table:
+----+------------------+
| id | email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
| 3  | john@example.com |
+----+------------------+
Output: 
+----+------------------+
| id | email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
+----+------------------+
Explanation: john@example.com is repeated two times. We keep the row with the smallest Id = 1.

In [67]:
email_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/029_DeleteDuplicateEmails.csv')

In [411]:
email_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- email: string (nullable = true)



In [467]:
win_spec = Window.partitionBy('email').orderBy(asc('id'))

In [470]:
email_df.withColumn('rnk',rank().over(win_spec)).filter(col('rnk') ==1).drop('rnk').show()

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
+---+----------------+



In [471]:
email_df.createOrReplaceTempView('emaildf')

In [465]:
spark.sql("select id,email from \
          (select id,email,rank() OVER (partition by email order by id asc) as rnk from emaildf) \
          where rnk = 1 ").show()

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
+---+----------------+



In [479]:
spark.sql(" SELECT min(id),email FROM emaildf GROUP BY Email ").show()

+-------+----------------+
|min(id)|           email|
+-------+----------------+
|      1|john@example.com|
|      2| bob@example.com|
+-------+----------------+



#### 030 Group Sold Products By The Date

Table Activities:

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| sell_date   | date    |
| product     | varchar |
+-------------+---------+
There is no primary key (column with unique values) for this table. It may contain duplicates.
Each row of this table contains the product name and the date it was sold in a market.
 

Write a solution to find for each date the number of different products sold and their names.

The sold products names for each date should be sorted lexicographically.

Return the result table ordered by sell_date.

The result format is in the following example.

 

Example 1:

Input: 
Activities table:
+------------+------------+
| sell_date  | product     |
+------------+------------+
| 2020-05-30 | Headphone  |
| 2020-06-01 | Pencil     |
| 2020-06-02 | Mask       |
| 2020-05-30 | Basketball |
| 2020-06-01 | Bible      |
| 2020-06-02 | Mask       |
| 2020-05-30 | T-Shirt    |
+------------+------------+
Output: 
+------------+----------+------------------------------+
| sell_date  | num_sold | products                     |
+------------+----------+------------------------------+
| 2020-05-30 | 3        | Basketball,Headphone,T-shirt |
| 2020-06-01 | 2        | Bible,Pencil                 |
| 2020-06-02 | 1        | Mask                         |
+------------+----------+------------------------------+
Explanation: 
For 2020-05-30, Sold items were (Headphone, Basketball, T-shirt), we sort them lexicographically and separate them by a comma.
For 2020-06-01, Sold items were (Pencil, Bible), we sort them lexicographically and separate them by a comma.
For 2020-06-02, the Sold item is (Mask), we just return it.

In [68]:
soldproduct_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/030_GroupSoldProductsByTheDate.csv')

In [69]:
soldproduct_df.printSchema()

root
 |-- sell_date: string (nullable = true)
 |-- product: string (nullable = true)



In [70]:
#collect_set,collect_list,concat_ws
soldproduct_df.groupBy('sell_date')\
        .agg(countDistinct('product').alias('num_sold'),collect_list('product').alias('product') ).show(truncate=False)

+----------+--------+--------------------------------+
|sell_date |num_sold|product                         |
+----------+--------+--------------------------------+
|2020-05-30|3       |[Headphone, Basketball, T-Shirt]|
|2020-06-01|2       |[Bible, Pencil]                 |
|2020-06-02|1       |[Mask, Mask]                    |
+----------+--------+--------------------------------+



In [535]:
soldproduct_df.groupBy('sell_date')\
        .agg(countDistinct('product').alias('num_sold'),concat_ws(',',collect_set('product')).alias('product') ).show(truncate=False)

+----------+--------+----------------------------+
|sell_date |num_sold|product                     |
+----------+--------+----------------------------+
|2020-05-30|3       |T-Shirt,Basketball,Headphone|
|2020-06-01|2       |Pencil,Bible                |
|2020-06-02|1       |Mask                        |
+----------+--------+----------------------------+



In [488]:
soldproduct_df.createOrReplaceTempView('soldproductdf')

In [512]:
spark.sql("select sell_date,count(distinct product) as num_sold,collect_list(product) as products \
        from soldproductdf group by sell_date ").show(truncate=False)

+----------+--------+--------------------------------+
|sell_date |num_sold|products                        |
+----------+--------+--------------------------------+
|2020-05-30|3       |[Headphone, Basketball, T-Shirt]|
|2020-06-01|2       |[Bible, Pencil]                 |
|2020-06-02|1       |[Mask, Mask]                    |
+----------+--------+--------------------------------+



####  031 List the Products Ordered in a Period

Table: Products

+------------------+---------+
| Column Name      | Type    |
+------------------+---------+
| product_id       | int     |
| product_name     | varchar |
| product_category | varchar |
+------------------+---------+
product_id is the primary key (column with unique values) for this table.
This table contains data about the company's products.
 

Table: Orders

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| order_date    | date    |
| unit          | int     |
+---------------+---------+
This table may have duplicate rows.
product_id is a foreign key (reference column) to the Products table.
unit is the number of products ordered in order_date.
 

Write a solution to get the names of products that have at least 100 units ordered in February 2020 and their amount.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Products table:
+-------------+-----------------------+------------------+
| product_id  | product_name          | product_category |
+-------------+-----------------------+------------------+
| 1           | Leetcode Solutions    | Book             |
| 2           | Jewels of Stringology | Book             |
| 3           | HP                    | Laptop           |
| 4           | Lenovo                | Laptop           |
| 5           | Leetcode Kit          | T-shirt          |
+-------------+-----------------------+------------------+
Orders table:
+--------------+--------------+----------+
| product_id   | order_date   | unit     |
+--------------+--------------+----------+
| 1            | 2020-02-05   | 60       |
| 1            | 2020-02-10   | 70       |
| 2            | 2020-01-18   | 30       |
| 2            | 2020-02-11   | 80       |
| 3            | 2020-02-17   | 2        |
| 3            | 2020-02-24   | 3        |
| 4            | 2020-03-01   | 20       |
| 4            | 2020-03-04   | 30       |
| 4            | 2020-03-04   | 60       |
| 5            | 2020-02-25   | 50       |
| 5            | 2020-02-27   | 50       |
| 5            | 2020-03-01   | 50       |
+--------------+--------------+----------+
Output: 
+--------------------+---------+
| product_name       | unit    |
+--------------------+---------+
| Leetcode Solutions | 130     |
| Leetcode Kit       | 100     |
+--------------------+---------+
Explanation: 
Products with product_id = 1 is ordered in February a total of (60 + 70) = 130.
Products with product_id = 2 is ordered in February a total of 80.
Products with product_id = 3 is ordered in February a total of (2 + 3) = 5.
Products with product_id = 4 was not ordered in February 2020.
Products with product_id = 5 is ordered in February a total of (50 + 50) = 100.

In [71]:
order_Schama = sold_schema =  StructType([
                                (StructField('product_id',IntegerType(),False)),
                                (StructField('order_date',DateType(),False)),
                                (StructField('unit',IntegerType(),False))
                            ])

In [72]:
product_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/031_ListtheProductsOrderedinaPeriod.csv')

In [73]:
order_df = spark.read.option('header',True).schema(order_Schama).format('csv')\
        .load('../data/easymedium/031a_ListtheProductsOrderedinaPeriod.csv')\
        .withColumn('oproduct_id',col('product_id')).withColumn('order_date',date_format(col('order_date'),'yyyy-MM'))

In [40]:
product_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category: string (nullable = true)



In [41]:
order_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- unit: integer (nullable = true)
 |-- oproduct_id: integer (nullable = true)



In [51]:
order_df.join(product_df,product_df.product_id == order_df.oproduct_id)\
    .select('product_name','unit').filter(col('order_date') == lit('2020-02'))\
    .groupby(col('product_name')).agg(sum(col('unit'))).filter(col('sum(unit)') >= 100).show()


+------------------+---------+
|      product_name|sum(unit)|
+------------------+---------+
|      Leetcode Kit|      100|
|Leetcode Solutions|      130|
+------------------+---------+



In [52]:
product_df.createOrReplaceTempView('productdf')
order_df.createOrReplaceTempView('orderdf')

In [56]:
spark.sql("select product_name,sum(unit) as total_units from orderdf join productdf on productdf.product_id = oproduct_id \
        where order_date = '2020-02' group by product_name having total_units >= 100").show()

+------------------+-----------+
|      product_name|total_units|
+------------------+-----------+
|      Leetcode Kit|        100|
|Leetcode Solutions|        130|
+------------------+-----------+



#### 032 Find Users With Valid E-Mails

Table: Users

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| name          | varchar |
| mail          | varchar |
+---------------+---------+
user_id is the primary key (column with unique values) for this table.
This table contains information of the users signed up in a website. Some e-mails are invalid.
 

Write a solution to find the users who have valid emails.

A valid e-mail has a prefix name and a domain where:

The prefix name is a string that may contain letters (upper or lower case), digits, underscore '_', period '.', and/or dash '-'. The prefix name must start with a letter.
The domain is '@leetcode.com'.
Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Users table:
+---------+-----------+-------------------------+
| user_id | name      | mail                    |
+---------+-----------+-------------------------+
| 1       | Winston   | winston@leetcode.com    |
| 2       | Jonathan  | jonathanisgreat         |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
| 5       | Marwan    | quarz#2020@leetcode.com |
| 6       | David     | david69@gmail.com       |
| 7       | Shapiro   | .shapo@leetcode.com     |
+---------+-----------+-------------------------+
Output: 
+---------+-----------+-------------------------+
| user_id | name      | mail                    |
+---------+-----------+-------------------------+
| 1       | Winston   | winston@leetcode.com    |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
+---------+-----------+-------------------------+
Explanation: 
The mail of user 2 does not have a domain.
The mail of user 5 has the # sign which is not allowed.
The mail of user 6 does not have the leetcode domain.
The mail of user 7 starts with a period.

In [74]:
validuser_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/032_FindUsersWithValidEMails.csv')

In [91]:
validuser_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- mail: string (nullable = true)



In [95]:
validuser_df.select('user_id','name','mail').filter( (col('mail').like('%@leetcode.com')) \
                            & (~col('mail').like('%#%')) & (~col('mail').like('.%'))  ).show()

+-------+---------+--------------------+
|user_id|     name|                mail|
+-------+---------+--------------------+
|      1|  Winston|winston@leetcode.com|
|      3|Annabelle| bella-@leetcode.com|
|      4|    Sally|sally.come@leetco...|
+-------+---------+--------------------+



In [94]:
validuser_df.createOrReplaceTempView('validuserdf')

In [90]:
spark.sql("select user_id,name,mail from validuserdf where mail like '%@leetcode.com' \
        and mail not like '%#%' and mail not like '.%' ").show()

+-------+---------+--------------------+
|user_id|     name|                mail|
+-------+---------+--------------------+
|      1|  Winston|winston@leetcode.com|
|      3|Annabelle| bella-@leetcode.com|
|      4|    Sally|sally.come@leetco...|
+-------+---------+--------------------+

